# Fetching Oregon Parcel Data
This Notebook demonstrates how to access Web Services employed by the ORMAP web application to retrieve parcel (tax lot) data from Oregon properties. As configured by default, this workflow will employ a multithreaded download to fetch all the parcels in the parcel database in 1000-parcel chunks. 

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, Polygon

import requests

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

## Workhorse API Call
The `parcels_from_ormap` function below is extensible to a variety of other use cases. 

In [ ]:
def parcels_from_ormap(bbox, inSR=4326, features=True, **kwargs):
    """
    Returns tax lot boundaries as features from ORMAP's parcel database.
    
    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy).
    inSR : int
      spatial reference for bounding box, such as an EPSG code (e.g., 4326)
    features : bool
      whether to return features instead of the JSON of the request
    
    Returns
    -------
    gdf : GeoDataFrame
      features in vector format
    jsn : dict
      JSON-formatted request, returned when features=False
    r : requests.request
      a request object, returned if the web request fails
    """
    BASE_URL = ''.join([
        'https://utility.arcgis.com/usrsvcs/servers/',
        '78bbb0d0d9c64583ad5371729c496dcc/rest/services/',
        'Secure/DOR_ORMAP/MapServer/3/query?',
    ])

    params = dict(f='geojson',
                  returnGeometry='true',
                  spatialRel='esriSpatialRelIntersects',
                  geometry=(f'{{"xmin":{bbox[0]},"ymin":{bbox[1]},'
                            f'"xmax":{bbox[2]},"ymax":{bbox[3]},'
                            f'"spatialReference":{{"wkid":{inSR}}}}}'),
                  geometryType='esriGeometryEnvelope',
                  outFields='*',
                  outSR=inSR)
    for key, value in kwargs.items():
        params.update({key: value})

    try:
        r = requests.get(BASE_URL, params=params, headers={'referer': 'https://ormap.net/'})
        jsn = r.json()
    
        if features:
            if len(jsn['features']) == 0:
                gdf = gpd.GeoDataFrame(geometry=[Polygon()], crs=inSR)
            else:
                gdf = gpd.GeoDataFrame.from_features(jsn, crs=inSR).sort_values(by='OBJECTID')
                gdf['geometry'] = gdf.buffer(0)

            return gdf
        else:
            return jsn
    except:
        return r

## Oregon is our Area of Interest
Here we define a bounding box that should encapsulate the entire State of Oregon. The web request above will only return parcels that intersect the user-specified bounding box. We do not currently want to filter out any of the parcels based on the bounding box, so we use the entire State as our AOI.

In [ ]:
OR_BBOX = [-125.068359,41.656497,-116.147461,46.483265]
EPSG = 4326

By specifying the optional keyword argument `returnIdsOnly=True` and `features=False`, we will receive a JSON response that includes all the object IDs for parcels that are included in the database. We can use this to iterate through subsequent requests, limiting each request to stay within the maximum number of features allowed per request (1,000).

In [ ]:
ids = np.array(parcels_from_ormap(OR_BBOX, inSR=EPSG, returnIdsOnly=True, features=False)['objectIds'])

Here we write a simple helper function to fetch parcels starting at ID `start_id` plus the next `step` size number of parcels. We set this up to default to retrieve 1000 parcels at a time. You can modify that path specified in `outfile` below to redirect the saving of the GeoJSON files to wherever you like.

In [ ]:
def fetch_save_parcels(start_id, step=1000, overwrite=False, out_dir='.'):
    fname = f'parcels_{start_id}-{start_id+step-1}.geojson'
    outfile = os.path.join(out_dir, fname)
    
    if not os.path.exists(outfile) or overwrite:
        with warnings.catch_warnings():
            warnings.simplefilter(action='ignore', category=FutureWarning)

            f = parcels_from_ormap(OR_BBOX, inSR=EPSG, 
                                   where=f'OBJECTID >= {start_id} AND OBJECTID < {start_id + step}')

            f.to_file(outfile, driver='GeoJSON')
    return

Here we implement a multi-threaded download to fetch and save all the available parcel data from the ORMAP database one chunk at a time.

In [ ]:
steps = range(1,ids[-1]+1,1000)

with tqdm(total=len(steps)) as pbar:
    with ThreadPoolExecutor(12) as executor:
        jobs = [executor.submit(fetch_save_parcels, step) for step in steps]
        for job in as_completed(jobs):
            pbar.update()